**0. Imports**

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
# from typing import List
import os

**1. RT Sampling Points**
- 10 - 15 with 100 intervals
- Steps should be 0.05 but with variation of 0.06 and 0.04. In the end should add up to the end ofthe range.

In [2]:
# Function to create a linspace with variation (can change variation)
def rt_variable_linspace(start: float, end: float, num_steps: int, steps_variation: float = 0.05) -> [float]:
    if num_steps < 2:
        raise ValueError("num_steps must be at least 2")

    # Generate regular spaced values
    regular = np.linspace(start, end, num_steps)
    steps = np.diff(regular)
    variation = np.mean(steps) * steps_variation

    # Generate noise for internal steps (excluding first and last)
    noise = np.random.uniform(-variation, variation, num_steps - 2)
    noise -= np.mean(noise)  # zero-sum 
    # noise = float(noise)  # Convert to float for consistent addition

    # Apply noise to internal steps
    modified_steps = steps.copy()
    modified_steps[0:-1] = steps[0:-1] + noise
    modified_steps[0:-1] += noise

    # Reconstruct the irregular range
    irregular = [start]
    for step in modified_steps:
        irregular.append(irregular[-1] + step)

    return irregular

rt_sampling_points = rt_variable_linspace(10, 15, 100, 0.1)
rt_sampling_points[-10:]  # Show the last 10 points

[14.55164888623068,
 14.59635631019129,
 14.639668147713232,
 14.683005059305243,
 14.738483186595055,
 14.79278805412151,
 14.84134402528028,
 14.898746643500886,
 14.949494949494943,
 14.999999999999993]

**2. MZ Sampling Rate**
- 150 - 160 with variable intervals
- Steps should be 0.05 but with variation of 0.06 and 0.04. In the end should add up to the end ofthe range.

In [3]:
# Function to create a linspace with variation (can change variation)
def mz_variable_linspace(start_mz: int, end_mz: int, min_steps: int, 
                         max_steps: int, steps_variation: float = 0.05):
    if min_steps < 2:
        raise ValueError("min_steps must be at least 2")
    
    # Randomly choose n_intervals within range
    num_steps = np.random.randint(min_steps, max_steps)
    print(num_steps)
   
    # Generate regular spaced values
    regular = np.linspace(start_mz, end_mz, num_steps)
    steps = np.diff(regular)
    variation = np.mean(steps) * steps_variation

    # Generate noise for internal steps (excluding first and last)
    noise = np.random.uniform(-variation, variation, num_steps - 2)
    noise -= np.mean(noise)  # zero-sum constraint

    # Apply noise to internal steps
    modified_steps = steps.copy()
    modified_steps[0:-1] = steps[0:-1] + noise
    modified_steps[0:-1] += noise

    # Reconstruct the irregular range
    irregular = [start_mz]
    for step in modified_steps:
        irregular.append(irregular[-1] + step)
    
    return irregular

**3. Combine into Matrix (Grid)**

In [4]:
def create_grid(rt: np.ndarray):
    combined_matrix = pd.DataFrame({"rt": rt})

    mz_column = []
    for rt_val in combined_matrix["rt"]:
        # Generate mz array for each rt
        mz_array = mz_variable_linspace(start_mz=150, end_mz=160, min_steps=990, max_steps=1010, steps_variation=0.1)
        mz_column.append(mz_array)

    combined_matrix["mz"] = mz_column
    return combined_matrix

grid = create_grid(rt=rt_sampling_points)

1005
1001
1008
1008
992
1001
1000
994
1009
1001
1007
1006
999
993
999
1008
998
1002
993
995
1001
1008
1005
1002
995
1003
997
1007
994
1009
1009
996
1001
991
994
1006
1004
999
1005
994
1002
1000
999
999
996
1008
1000
1005
1004
995
994
1002
992
990
992
990
990
994
996
1000
993
1001
994
1003
1009
993
1008
990
995
993
996
993
990
1008
1007
1002
1004
1005
990
1006
1004
1000
1003
994
1000
995
997
996
991
991
994
1008
1000
996
1001
993
990
993
1007
994


**4. Export Grid**

In [5]:
def save_grid(df, base_filename = "v2_grid"):
    # Start with the base filename
    filename = f"{base_filename}.json"
    counter = 1
    
    # Check if the file already exists and increment the counter if it does
    while os.path.exists(filename):
        filename = f"{base_filename}_{counter}.json"
        counter += 1
    
    # Save the dataframe to the unique filename
    df.to_json(filename, index=False)
    print(f"File saved as: {filename}")

save_grid(grid)

File saved as: v2_grid.json


**5. Plot Grid**

In [6]:
import plotly.express as px

# Prepare data for scatterplot
scatter_data = grid.explode('mz')  # Explode the 'mz' column to create individual rows for each mz value

# Create scatterplot
fig = px.scatter(scatter_data, x='rt', y='mz', title='Grid Scatterplot', labels={'rt': 'RT', 'mz': 'MZ'})
fig.update_layout(
    width=1200,
    height=800,
    template="plotly_white"  # Use a black-and-white background
)
# Show the plot
fig.update_layout(
    # xaxis=dict(range=[xmin, xmax]),
    yaxis=dict(range=[150, 150.5])
)
fig.update_traces(marker=dict(size=2, opacity=0.8))  # Adjust marker size and opacity
fig.update_xaxes(title_text='RT', title_font=dict(size=14))
fig.update_yaxes(title_text='MZ', title_font=dict(size=14))
fig.update_layout(
    title='RT vs MZ Scatterplot',
    title_x=0.5,  # Center the title
    title_font=dict(size=20),
    font=dict(size=12)
)
# Show the plot
fig.show()